# CrossValidation-S50F-DNN & Optimize Tune Epcoch and BatchSize


Baseline Neural Network Model Performance



In [1]:
xseed=38

from numpy.random import seed
seed(xseed)
from tensorflow import set_random_seed
set_random_seed(xseed)

ModuleNotFoundError: No module named 'tensorflow'

In [378]:
 # Binary Classification with Sonar Dataset: Baseline
import numpy
import pandas


from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras import models

from keras.utils import np_utils
from keras.utils import to_categorical



from keras.layers import Dropout
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.optimizers import Adadelta
from keras.constraints import maxnorm


from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline





from keras.utils import np_utils

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import LabelEncoder




import joblib

import os
from datetime import datetime

In [379]:
st='2007'
ed='2020'



In [380]:
filename='ML-Long_S50M15_07ToNow-Train.csv'
filepath=os.path.abspath(f'D:\Dev-Trong\DataSet\/{filename}')
print(filepath)


D:\Dev-Trong\DataSet\ML-Long_S50M15_07ToNow-Train.csv


# Train Data by WFA

In [381]:
# load dataset

dataframe = pandas.read_csv(filepath,index_col="datetime", parse_dates=['datetime'],dayfirst=True)

print(dataframe.shape)
              
dataframe.info()

(70693, 22)
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 70693 entries, 2007-01-03 09:45:00 to 2020-06-10 16:45:00
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   open                    70693 non-null  float64
 1   high                    70693 non-null  float64
 2   low                     70693 non-null  float64
 3   close                   70693 non-null  float64
 4   indy_ma-550             70693 non-null  float64
 5   indy_ma-1100            70693 non-null  float64
 6   indy_hh-550             70693 non-null  float64
 7   indy_ll-550             70693 non-null  float64
 8   indy_mid-550            70693 non-null  float64
 9   indy_hh2-1100           70693 non-null  float64
 10  indy_ll2-1100           70693 non-null  float64
 11  indy_mid2-1100          70693 non-null  float64
 12  indy_macd110-440        70693 non-null  float64
 13  indy_signal110-440-110  70693 non-null  floa

In [382]:
df=dataframe[st:ed]
#dfOHLC=df.loc[:,['open','high','low','close']]
df.drop(columns=['open','high','low','close'],inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 70693 entries, 2007-01-03 09:45:00 to 2020-06-10 16:45:00
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   indy_ma-550             70693 non-null  float64
 1   indy_ma-1100            70693 non-null  float64
 2   indy_hh-550             70693 non-null  float64
 3   indy_ll-550             70693 non-null  float64
 4   indy_mid-550            70693 non-null  float64
 5   indy_hh2-1100           70693 non-null  float64
 6   indy_ll2-1100           70693 non-null  float64
 7   indy_mid2-1100          70693 non-null  float64
 8   indy_macd110-440        70693 non-null  float64
 9   indy_signal110-440-110  70693 non-null  float64
 10  indy_hist_macd110-440   70693 non-null  float64
 11  indy_rsi25-ma20         70693 non-null  float64
 12  indy_6ATRTrail_DC-110   70693 non-null  float64
 13  cate_3trend-550_ma110   70693 non-null  int64  
 14  cat

In [383]:
labelName=df.columns[-1]
labelName

'LongSignal'

In [384]:
X_df=df.drop(columns=labelName)
y_df=df[labelName]

In [385]:
print(X_df.info())
X_df.tail(3)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 70693 entries, 2007-01-03 09:45:00 to 2020-06-10 16:45:00
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   indy_ma-550             70693 non-null  float64
 1   indy_ma-1100            70693 non-null  float64
 2   indy_hh-550             70693 non-null  float64
 3   indy_ll-550             70693 non-null  float64
 4   indy_mid-550            70693 non-null  float64
 5   indy_hh2-1100           70693 non-null  float64
 6   indy_ll2-1100           70693 non-null  float64
 7   indy_mid2-1100          70693 non-null  float64
 8   indy_macd110-440        70693 non-null  float64
 9   indy_signal110-440-110  70693 non-null  float64
 10  indy_hist_macd110-440   70693 non-null  float64
 11  indy_rsi25-ma20         70693 non-null  float64
 12  indy_6ATRTrail_DC-110   70693 non-null  float64
 13  cate_3trend-550_ma110   70693 non-null  int64  
 14  cat

,indy_ma-550,indy_ma-1100,indy_hh-550,indy_ll-550,indy_mid-550,indy_hh2-1100,indy_ll2-1100,indy_mid2-1100,indy_macd110-440,indy_signal110-440-110,indy_hist_macd110-440,indy_rsi25-ma20,indy_6ATRTrail_DC-110,cate_3trend-550_ma110,cate_2trend-1100_ma220,cate_rannkHL1100-ma66,cate_CombineTrend
datetime,,,,,,,,,,,,,,,,,
2020-06-10 16:15:00,887.68,847.74,977.9,834.2,906.05,977.9,715.3,846.6,56.43,41.48,14.95,46.25,952.82,3,1,4,4
2020-06-10 16:30:00,887.85,847.94,977.9,834.2,906.05,977.9,715.3,846.6,56.54,41.78,14.76,46.64,952.65,3,1,4,4
2020-06-10 16:45:00,888.02,848.15,977.9,834.2,906.05,977.9,715.3,846.6,56.64,42.07,14.57,47.13,952.71,3,1,4,4


In [386]:

y_df.tail(5)

datetime
2020-06-10 15:45:00    1
2020-06-10 16:00:00    1
2020-06-10 16:15:00    1
2020-06-10 16:30:00    1
2020-06-10 16:45:00    1
Name: LongSignal, dtype: int64

In [387]:
# split into input (X) and output (Y) variables
X = X_df.to_numpy()
y =y_df.to_numpy()


In [388]:
#from keras.utils import to_categorical
if True :
 encoder = LabelEncoder()
 encoder.fit(y)
 y = encoder.transform(y)
 # convert integers to dummy variables (i.e. one hot encoded)
 y = to_categorical(y)

y[:10]

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [389]:
print(X.shape)
print(y.shape)

(70693, 17)
(70693, 2)


In [390]:
h1=17
h2=17
h3=0

input_n=X.shape[1]

xpoch=105
xbatch=32

#my_kernelInit='he_uniform'
#kernel_initializer="glorot_uniform"  #default

In [391]:
def BasicModel():

  model = Sequential()  
  
  model.add(Dense(h1, input_dim=X.shape[1], kernel_initializer='he_normal',activation='relu' ))
  model.add(Dense(h2, kernel_initializer='he_normal',activation='relu'))

  
  model.add(Dense(1,  activation='sigmoid'))
  
    # Compile model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model  



In [392]:
def ModelByAllTuning():
    model_x=Sequential()
    
    param_dropout=0.02
    vaMaxNorm_WC=4
    
    #l2_val=0.01 #0.865 vs 0.851
    l2_val=0.00100000 #0.93 vs 0.84
    
     #input
    model_x.add(Dropout(param_dropout, input_shape=(input_n,)))
    
    #layer#1
    model_x.add(Dense(h1, activation="relu", kernel_constraint=maxnorm(vaMaxNorm_WC),kernel_regularizer=l2(l2_val)))
    model_x.add(Dropout(param_dropout))
    
    #layer#2
    model_x.add(Dense(h2, activation="relu", kernel_constraint=maxnorm(vaMaxNorm_WC),kernel_regularizer=l2(l2_val)))
    model_x.add(Dropout(param_dropout))
    

    #model_x.add(Dense(1, activation="sigmoid"))
    #model_x.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    model_x.add(Dense(2, activation='softmax'))
    model_x.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model_x

In [393]:
def CreatePipeLine(fn,n_epochs,n_batch_size):
 #early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=1, verbose=0, mode='auto')
 estimator_steps=[]
 #estimator_steps.append(('standardize', StandardScaler()))
 estimator_steps.append(('minmax_range', MinMaxScaler()))
 #print('Select Model funciton: ',fn)
 estimator_steps.append(('mlp', KerasClassifier(build_fn=fn, epochs=n_epochs, batch_size=n_batch_size)))

 xpipeline=Pipeline(estimator_steps)
 return xpipeline 

In [401]:

xsplits=5

In [402]:
option=2
if option==1:
 #xfold = StratifiedKFold(n_splits=xsplits, shuffle=True, random_state=xseed)
 xfold = KFold(n_splits=xsplits, shuffle=True, random_state=xseed)
else :
 
 xfold = TimeSeriesSplit(n_splits=xsplits,max_train_size=None)
 df_Xtime=(X_df.reset_index())[['datetime']]
 print('train from ',df_Xtime.loc[0,'datetime'] ,' to test as follows')
 for train, test in xfold.split(X_df):
  print('test from ',df_Xtime.loc[test[0],'datetime']) 




train from  2007-01-03 09:45:00  to test as follows
test from  2009-04-17 10:30:00
test from  2011-08-11 15:45:00
test from  2013-11-20 12:15:00
test from  2016-02-10 09:45:00
test from  2018-04-05 15:30:00


In [403]:
mypipeline=CreatePipeLine(ModelByAllTuning,xpoch,xbatch)
 
results = cross_val_score(mypipeline, X, y, cv=xfold)
    

Epoch 1/105
11783/11783 [==============================] - 1s 56us/step - loss: 0.3769 - accuracy: 0.8494
Epoch 2/105
11783/11783 [==============================] - 0s 37us/step - loss: 0.2414 - accuracy: 0.9084
Epoch 3/105
11783/11783 [==============================] - 0s 37us/step - loss: 0.2162 - accuracy: 0.9152
Epoch 4/105
11783/11783 [==============================] - 0s 38us/step - loss: 0.2038 - accuracy: 0.9187 0s - loss: 0.2072 - ac
Epoch 5/105
11783/11783 [==============================] - 0s 38us/step - loss: 0.1916 - accuracy: 0.9209
Epoch 6/105
11783/11783 [==============================] - 0s 37us/step - loss: 0.1890 - accuracy: 0.9208
Epoch 7/105
11783/11783 [==============================] - 0s 39us/step - loss: 0.1789 - accuracy: 0.9276 0s - loss: 0.1816 - ac
Epoch 8/105
11783/11783 [==============================] - 0s 38us/step - loss: 0.1753 - accuracy: 0.9293
Epoch 9/105
11783/11783 [==============================] - 0s 38us/step - loss: 0.1768 - accuracy: 0.9289


23565/23565 [==============================] - 1s 38us/step - loss: 0.2085 - accuracy: 0.9193 0s - loss: 0.2
Epoch 37/105
23565/23565 [==============================] - 1s 37us/step - loss: 0.2086 - accuracy: 0.9207
Epoch 38/105
23565/23565 [==============================] - ETA: 0s - loss: 0.2038 - accuracy: 0.92 - 1s 36us/step - loss: 0.2041 - accuracy: 0.9222
Epoch 39/105
23565/23565 [==============================] - 1s 37us/step - loss: 0.2079 - accuracy: 0.9212 0s - loss: 0
Epoch 40/105
23565/23565 [==============================] - 1s 36us/step - loss: 0.2051 - accuracy: 0.9230
Epoch 41/105
23565/23565 [==============================] - 1s 36us/step - loss: 0.2049 - accuracy: 0.9218
Epoch 42/105
23565/23565 [==============================] - 1s 36us/step - loss: 0.2039 - accuracy: 0.9239
Epoch 43/105
23565/23565 [==============================] - 1s 36us/step - loss: 0.2025 - accuracy: 0.9231
Epoch 44/105
23565/23565 [==============================] - 1s 36us/step - loss: 0.2036

Epoch 102/105
23565/23565 [==============================] - 1s 37us/step - loss: 0.1774 - accuracy: 0.9356
Epoch 103/105
23565/23565 [==============================] - 1s 36us/step - loss: 0.1706 - accuracy: 0.9409
Epoch 104/105
23565/23565 [==============================] - 1s 36us/step - loss: 0.1722 - accuracy: 0.9388 0s -
Epoch 105/105
11782/11782 [==============================] - 0s 31us/step
Epoch 1/105
35347/35347 [==============================] - 2s 43us/step - loss: 0.4158 - accuracy: 0.8349
Epoch 2/105
35347/35347 [==============================] - 1s 37us/step - loss: 0.3516 - accuracy: 0.8615 0s - loss: 0.3546 
Epoch 3/105
35347/35347 [==============================] - 1s 37us/step - loss: 0.3384 - accuracy: 0.8660
Epoch 4/105
35347/35347 [==============================] - 1s 37us/step - loss: 0.3269 - accuracy: 0.8732 0s - loss: 0.3
Epoch 5/105
35347/35347 [==============================] - 1s 37us/step - loss: 0.3203 - accuracy: 0.8737
Epoch 6/105
35347/35347 [========

47129/47129 [==============================] - 2s 37us/step - loss: 0.2668 - accuracy: 0.8935 0s - loss: - ETA: 0s - loss: 0.2691 - accu
Epoch 26/105
47129/47129 [==============================] - 2s 37us/step - loss: 0.2635 - accuracy: 0.8969
Epoch 27/105
47129/47129 [==============================] - ETA: 0s - loss: 0.2673 - accuracy: 0.8937 ETA: 0s - loss: 0.2664 - accuracy: 0. - 2s 37us/step - loss: 0.2667 - accuracy: 0.8941
Epoch 28/105
47129/47129 [==============================] - 2s 37us/step - loss: 0.2614 - accuracy: 0.8972
Epoch 29/105
47129/47129 [==============================] - 2s 37us/step - loss: 0.2625 - accuracy: 0.8971 0s - loss: 0.2600 - ac
Epoch 30/105
47129/47129 [==============================] - 2s 37us/step - loss: 0.2607 - accuracy: 0.8983
Epoch 31/105
47129/47129 [==============================] - 2s 37us/step - loss: 0.2576 - accuracy: 0.8987
Epoch 32/105
47129/47129 [==============================] - 2s 38us/step - loss: 0.2570 - accuracy: 0.8999 1s - loss

58911/58911 [==============================] - 2s 41us/step - loss: 0.2760 - accuracy: 0.8904 0s - loss: 0.2776 - accuracy: 0.88 - ETA: 0s - loss: 0.277
Epoch 44/105
58911/58911 [==============================] - 2s 38us/step - loss: 0.2752 - accuracy: 0.8904 0s - loss: 0.2752 - accuracy: 0.
Epoch 45/105
58911/58911 [==============================] - 2s 37us/step - loss: 0.2733 - accuracy: 0.8916
Epoch 46/105
58911/58911 [==============================] - 2s 38us/step - loss: 0.2717 - accuracy: 0.8927
Epoch 47/105
58911/58911 [==============================] - 2s 38us/step - loss: 0.2739 - accuracy: 0.8918 0s
Epoch 48/105
58911/58911 [==============================] - 2s 38us/step - loss: 0.2730 - accuracy: 0.8914
Epoch 49/105
58911/58911 [==============================] - 2s 38us/step - loss: 0.2726 - accuracy: 0.8915
Epoch 50/105
58911/58911 [==============================] - 2s 38us/step - loss: 0.2730 - accuracy: 0.8918
Epoch 51/105
58911/58911 [==============================] - 2s

In [404]:
print("Standardized:mean= %.2f%% std=(%.2f%%)" % (results.mean()*100, results.std()*100))
print("Standardized:max= %.2f%% min=%.2f%%" % (results.max()*100, results.min()*100))

Standardized:mean= 76.77% std=(5.60%)
Standardized:max= 85.54% min=70.60%


# Optimize Tune Epcoch and BatchSize

skfold = StratifiedKFold(n_splits=xsplits, shuffle=True, random_state=xseed)

#listEpochs=[50,100,150,300]
#listBatchSize=[1,20,32,64,len(X)]
listBatchSize=[10,20,32,64]
listResults=[]

#for n_epochs in listEpochs:
for n_bs in listBatchSize:
    
 #print("Read DNN=",n_epochs)   
 print("Batch DNN=",n_bs)

 #mypipeline=CreatePipeLine(create_dnn_model1,n_epochs,xbatch)
 mypipeline=CreatePipeLine(create_dnn_model1,xepoch,n_bs)
 
 results = cross_val_score(mypipeline, X, y, cv=skfold)
 listResults.append(results)
 print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    
    
for results in listResults:
 print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
